In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
from text_untils import *
import pickle
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
with open('output/bm25/bm25_20k_temp/bm25plus.pkl', 'rb') as f:
    bm25plus = pickle.load(f)

In [11]:
def load_docs_from_file(file_path):
    docs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                doc = json.loads(line)
                docs.append(doc)
            except json.JSONDecodeError:
                continue

    return docs

docs = load_docs_from_file('output/bm25/bm25_20k_temp/docs.jsonl')

In [12]:
query = "Đi xe máy vi phạm nồng độ cồn bị phạt bao nhiêu tiền?"
query = clean_text(query)
query = word_segment(query)
query = remove_stopword(normalize_text(query))

query_bm25 = query.split()
print(query_bm25)

['đi', 'xe_máy', 'vi_phạm', 'nồng_độ', 'cồn', 'phạt', 'bao_nhiêu', 'tiền']


In [14]:
bm25_scores = bm25plus.get_scores(query_bm25)
top_n_indices = np.argsort(bm25_scores)[::-1][:100]
top_n_score = bm25_scores[top_n_indices]

top_n_documents = [docs[idx] for idx in top_n_indices]

In [15]:
for doc_dict, score in zip(top_n_documents, top_n_score):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Score: {score}")

ID: 666342eee485130f03f56758, Document: đi xe_máy vi_phạm nồng_độ cồn có_thể phạt bao_nhiêu tiền, Score: 53.196836040565216
ID: 666342ede485130f03f5542b, Document: vi_phạm nồng_độ cồn phạt bao_nhiêu tiền, Score: 51.702071248707455
ID: 666342ede485130f03f5541a, Document: điều_khiển xe_máy nồng_độ cồn phạt vi_phạm hành_chính bao_nhiêu tiền, Score: 50.32022206385315
ID: 666342eee485130f03f56759, Document: đi xe_máy vi_phạm nồng_độ cồn nộp phạt, Score: 50.16789653873717
ID: 666342ece485130f03f53211, Document: vi_phạm nồng_độ cồn đội nón bảo_hiểm xe_máy vi_phạm phạt bao_nhiêu tiền, Score: 49.5973241388517
ID: 666342eae485130f03f51196, Document: phạt tiền nồng_độ cồn lái xe_máy bao_nhiêu, Score: 49.340779204719574
ID: 666342eee485130f03f5675a, Document: đi xe_máy ký biên_bản phạt vi_phạm nồng_độ cồn phạt, Score: 48.65627660391159
ID: 666342ece485130f03f541e7, Document: vi_phạm nồng_độ cồn tạm xe_máy bao_nhiêu, Score: 48.582722727299924
ID: 666342ede485130f03f54352, Document: phạt nồng_độ cồn

In [16]:
# min_score = min(bm25_scores)
# max_score = max(bm25_scores)
#
# rescaled_scores = [10*(score - min_score)/(max_score-min_score) for score in bm25_scores]
# top100_rescaled_scores = np.sort(rescaled_scores)[::-1][:100]

max_score = max(top_n_score)
rescaled_scores = [10*score/max_score for score in top_n_score]

for i, (doc_dict, score) in enumerate(zip(top_n_documents, rescaled_scores)):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Rescaled Score: {score:.4f}")


ID: 666342eee485130f03f56758, Document: đi xe_máy vi_phạm nồng_độ cồn có_thể phạt bao_nhiêu tiền, Rescaled Score: 10.0000
ID: 666342ede485130f03f5542b, Document: vi_phạm nồng_độ cồn phạt bao_nhiêu tiền, Rescaled Score: 9.7190
ID: 666342ede485130f03f5541a, Document: điều_khiển xe_máy nồng_độ cồn phạt vi_phạm hành_chính bao_nhiêu tiền, Rescaled Score: 9.4593
ID: 666342eee485130f03f56759, Document: đi xe_máy vi_phạm nồng_độ cồn nộp phạt, Rescaled Score: 9.4306
ID: 666342ece485130f03f53211, Document: vi_phạm nồng_độ cồn đội nón bảo_hiểm xe_máy vi_phạm phạt bao_nhiêu tiền, Rescaled Score: 9.3234
ID: 666342eae485130f03f51196, Document: phạt tiền nồng_độ cồn lái xe_máy bao_nhiêu, Rescaled Score: 9.2751
ID: 666342eee485130f03f5675a, Document: đi xe_máy ký biên_bản phạt vi_phạm nồng_độ cồn phạt, Rescaled Score: 9.1465
ID: 666342ece485130f03f541e7, Document: vi_phạm nồng_độ cồn tạm xe_máy bao_nhiêu, Rescaled Score: 9.1326
ID: 666342ede485130f03f54352, Document: phạt nồng_độ cồn xe_máy bao_nhiêu,

In [17]:
# doc_scores = top100_rescaled_scores

In [18]:
query_sbert = query.replace('_', ' ')
print(query_sbert)

đi xe máy vi phạm nồng độ cồn phạt bao nhiêu tiền


In [19]:
df = pd.read_json('data/cleaned/temp.json', lines=True)

In [20]:
doc_ids = [doc_dict['id'] for doc_dict in top_n_documents]
selected_rows = df[df['_id'].isin(doc_ids)]

In [21]:
sentences_title = []
ids_title = []

for idx in selected_rows.index:
    sentences_title.append(selected_rows.at[idx, 'title'])
    ids_title.append(selected_rows.at[idx, '_id'])

In [22]:
# model_title = SentenceTransformer('keepitreal/vietnamese-sbert')
model_title = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
embeddings_title = model_title.encode(sentences_title)

No sentence-transformers model found with name C:\Users\Admin/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.


In [23]:
query_embedding_title = model_title.encode([query_sbert], convert_to_tensor=True)
query_embedding_title = query_embedding_title.cpu()

In [24]:
similarities_title = cosine_similarity(query_embedding_title, embeddings_title)

top_50_indices_title = np.argsort(similarities_title[0])[::-1][:50]

for idx in top_50_indices_title:
    similarity_score = similarities_title[0][idx]
    similar_sentence = sentences_title[idx]
    doc_id = ids_title[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

ID: 666342eee485130f03f56758, Similarity Score: 0.9819, Sentence: đi xe máy vi phạm nồng độ cồn có thể phạt bao nhiêu tiền
ID: 666342eee485130f03f569fe, Similarity Score: 0.9565, Sentence: điều khiển xe máy nồng độ cồn phép phạt bao nhiêu
ID: 666342ebe485130f03f52554, Similarity Score: 0.9549, Sentence: lỗi vi phạm nồng độ cồn điều khiển xe máy tham gia giao thông phạt bao nhiêu tiền
ID: 666342ede485130f03f54352, Similarity Score: 0.9534, Sentence: phạt nồng độ cồn xe máy bao nhiêu
ID: 666342eee485130f03f563e3, Similarity Score: 0.9384, Sentence: lái xe máy vi phạm quy định nồng độ cồn phạt bao nhiêu tiền
ID: 666342ede485130f03f5541a, Similarity Score: 0.9311, Sentence: điều khiển xe máy nồng độ cồn phạt vi phạm hành chính bao nhiêu tiền
ID: 666342eae485130f03f517c8, Similarity Score: 0.9302, Sentence: điều khiển xe máy vi phạm nồng độ cồn xử phạt thế nào
ID: 666342eee485130f03f563f8, Similarity Score: 0.9197, Sentence: lái xe máy nồng độ cồn bao nhiêu phạt
ID: 666342eae485130f03f51196

In [25]:
sentences_anwser = []
ids_anwser = []

for idx in selected_rows.index:
    sentences_anwser.append(selected_rows.at[idx, 'answer'])
    ids_anwser.append(selected_rows.at[idx, '_id'])

In [26]:
# model_anwser = SentenceTransformer('keepitreal/vietnamese-sbert')
model_anwser = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
embeddings_anwser = model_anwser.encode(sentences_anwser)

No sentence-transformers model found with name C:\Users\Admin/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.


In [27]:
query_embedding_anwser = model_anwser.encode([query_sbert], convert_to_tensor=True)
query_embedding_anwser = query_embedding_anwser.cpu()

In [28]:
similarities_anwser = cosine_similarity(query_embedding_anwser, embeddings_anwser)

top_50_indices_anwser = np.argsort(similarities_anwser[0])[::-1][:50]

for idx in top_50_indices_anwser:
    similarity_score = similarities_anwser[0][idx]
    similar_sentence = sentences_anwser[idx]
    doc_id = ids_anwser[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

ID: 666342ede485130f03f5542b, Similarity Score: 0.7687, Sentence: căn cứ 5 nghị định 100 2019 nđ cp 6 nghị định 100 2019 nđ cp sửa đổi bổ sung nghị định 123 2021 nđ cp quy định xử phạt vi phạm nồng độ cồn đối với xe máy xe ô tô tuỳ thuộc nồng độ cồn đo 100 mililít máu 1 lít khí thở cụ thể xử phạt vi phạm nồng độ cồn đối với xe máy phạt tiền 2 3 triệu đồng nồng độ cồn 50 miligam 100 mililít máu 0 25 miligam 1 lít khí thở tước giấy phép lái xe 10 12 phạt tiền 4 5 triệu đồng nồng độ cồn 50 miligam 80 miligam 100 mililít máu 0 25 miligam 0 4 miligam 1 lít khí thở tước giấy phép lái xe 16 18 phạt tiền 6 8 triệu đồng nồng độ cồn 80 miligam 100 mililít máu 0 4 miligam 1 lít khí thở tước giấy phép lái xe 22 24 xử phạt vi phạm nồng độ cồn xe ô tô phạt tiền 6 8 triệu đồng nồng độ cồn 50 miligam 100 mililít máu 0 25 miligam 1 lít khí thở tước giấy phép lái xe 10 12 phạt tiền 16 18 triệu đồng nồng độ cồn 50 miligam 80 miligam 100 mililít máu 0 25 miligam 0 4 miligam 1 lít khí thở tước giấy phép lá

In [29]:
df_title = pd.DataFrame({
    'doc_id': [ids_title[idx] for idx in top_50_indices_title],
    'similarity_score_title': [similarities_title[0][idx] for idx in top_50_indices_title],
    'sentence_title': [sentences_title[idx] for idx in top_50_indices_title]
})

df_anwser = pd.DataFrame({
    'doc_id': [ids_anwser[idx] for idx in top_50_indices_anwser],
    'similarity_score_anwser': [similarities_anwser[0][idx] for idx in top_50_indices_anwser],
})

df_merged = pd.merge(df_title, df_anwser, on='doc_id')
df_merged['sbert_score'] = df_merged['similarity_score_title'] * df_merged['similarity_score_anwser']

df_merged_sorted = df_merged.sort_values(by='sbert_score', ascending=False)
top_20_rows = df_merged_sorted.head(20)

In [30]:
print(top_20_rows)

                      doc_id  similarity_score_title  \
0   666342eee485130f03f569fe                0.956474   
10  666342ede485130f03f5542b                0.891302   
1   666342ebe485130f03f52554                0.954854   
7   666342ece485130f03f53b98                0.906564   
3   666342eee485130f03f563e3                0.938399   
4   666342ede485130f03f5541a                0.931123   
6   666342eae485130f03f51196                0.918222   
2   666342ede485130f03f54352                0.953375   
5   666342eee485130f03f563f8                0.919719   
19  666342eee485130f03f56362                0.861406   
9   666342ebe485130f03f52493                0.893614   
14  666342ede485130f03f552c0                0.885235   
11  666342eae485130f03f5190e                0.889961   
12  666342eee485130f03f55949                0.888150   
13  666342ece485130f03f53211                0.887127   
15  666342efe485130f03f56cf9                0.877481   
17  666342eae485130f03f50d35                0.86

In [31]:
df_bm25 = pd.DataFrame({
    'doc_id': [doc_dict['id'] for doc_dict in top_n_documents],
    'bm25_score': top_n_score
})

df_bm25_filtered = df_bm25[df_bm25['doc_id'].isin(top_20_rows['doc_id'])]
df_final = pd.merge(df_bm25_filtered, top_20_rows, on='doc_id')
df_final['final_score'] = df_final['sbert_score'] * df_final['bm25_score']
df_final_sorted = df_final.sort_values(by='final_score', ascending=False)
print(df_final_sorted.head(5))

                     doc_id  bm25_score  similarity_score_title  \
0  666342ede485130f03f5542b   51.702071                0.891302   
1  666342ede485130f03f5541a   50.320222                0.931123   
3  666342eae485130f03f51196   49.340779                0.918222   
7  666342ebe485130f03f52554   48.086450                0.954854   
5  666342ece485130f03f53b98   48.086795                0.906564   

                                      sentence_title  similarity_score_anwser  \
0            vi phạm nồng độ cồn phạt bao nhiêu tiền                 0.768670   
1  điều khiển xe máy nồng độ cồn phạt vi phạm hàn...                 0.715864   
3         phạt tiền nồng độ cồn lái xe máy bao nhiêu                 0.725446   
7  lỗi vi phạm nồng độ cồn điều khiển xe máy tham...                 0.706170   
5  xe máy chấp hành kiểm tra nồng độ cồn phạt bao...                 0.741180   

   sbert_score  final_score  
0     0.685118    35.422008  
1     0.666558    33.541348  
3     0.666120    32